# Running Modeling Equation on GEE Landsat 9 Image Collection 

## Import Packages and Initialize GEE

In [1]:
##Import required Google Earth Engine python packages and check if they work in python environment
import ee
#ee.Authenticate(force=True) # Uncomment this if it is first time with GEE or account linking is having issues
ee.Initialize()
import geetools
import geemap
import os
import time

In [2]:
#import the map module that allows for attaching images to an interactive map
Map = geemap.Map()

## Import Google Earth Engine Feature Collection Data

In [3]:
#Import the river boundary from the Google Earth Engine Server
#Call in river in from a vector file saved into Google Earth Engine
TN_River = ee.FeatureCollection("users/pjf927/TN_River_GERS_StudySite")
#Some function require geometry values to clip features
TN_RiverGeom = TN_River.geometry() 
#Generate a square boundary around the river study area
RiverBounds = TN_RiverGeom.bounds()

## Import Google Earth Engine Image Collection Data

In [4]:
#Call in Landsat 9 Level 2, Collection 2, Tier 1 dataset. Info found at: https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_L2
LS9_SR = (
    ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
    .filterBounds(TN_River) #Filter only swath grids that cover the TN River Boundary
    .select(['SR_B4', 'QA_PIXEL'],['Red', 'QA_PIXEL']) #Assign imagery bands new names. The modeling equation only uses the red band and the QA band to mask out land and clouds. You may have to modify this
    .filterDate('2021-10-30', '2024-07-24') # First date: 2021-10-31  Last date: On going. Recomended to process only 5 year intervals due to length of the data
    .sort('system:time_start') #Sort collection by acquisition time
)

#Get a count of all images filtered in the Landsat Surface Relectance Collection
LS9_count_raw = LS9_SR.size().getInfo()
print("Landsat 9 Images: ", LS9_count_raw)

Landsat 9 Images:  203


## Function of mosaicing images of the same date together 

In [5]:
#Function declaration to later use on interating through an image collection
def mergeByDate(imgCol):
    #Convert the image collection to a list.
    imgList = imgCol.toList(imgCol.size())
    
    # Driver function for mapping the unique dates
    def uniqueDriver(image):
        return ee.Image(image).date().format("YYYY-MM-dd")
    
    uniqueDates = imgList.map(uniqueDriver).distinct()

    # Driver function for mapping the moasiacs
    def mosaicDriver(date):
        date = ee.Date(date)
        
        image = (imgCol
               .filterDate(date, date.advance(1, "day"))
               .mosaic())
        
        return image.set(
                        "system:time_start", date.millis(), 
                        "system:id", date.format("YYYY-MM-dd"),
                        "Date", date.format("YYYY-MM-dd"))
    
    mosaicImgList = uniqueDates.map(mosaicDriver)
    
    return ee.ImageCollection(mosaicImgList)

## Run the mosaic by date function on the image collection

In [6]:
LS9_SR_Mosaic = mergeByDate(LS9_SR)

In [7]:
# Function declaration for classifying the QA band. QA band info can be found here on page 13: https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/media/files/LSDS-1619_Landsat8-9-Collection2-Level2-Science-Product-Guide-v6.pdf
def getQABits(image, start, end, newName):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start, end+1):
       pattern += pow(2, i)
    # Return a single band image of the extracted QA bits, giving the band a new name.
    return image.select([0], [newName]).bitwiseAnd(pattern).rightShift(start)

## Function of determining the percentage of clouds within the water body boundary feature class

In [8]:
# Function declaration to calculate the percentage of clouds within the water body boundary feature class
def cloud_mask(image):
    # Select the QA band.
    QA = image.select(['QA_PIXEL'])
    # Get the internal_cloud_algorithm_flag bit.
    cloud_only = getQABits(QA, 3,3, 'cloud_mask').eq(1)
    cloudiness = cloud_only.reduceRegion(
             reducer = ee.Reducer.mean(), 
             geometry = TN_RiverGeom, 
             scale = 30,
             maxPixels = 1e13).get("cloud_mask")
    percent = ee.Number(cloudiness).multiply(100)
    return image.set("CLOUD_COVER", percent)

## Run the cloud percentage function on the image collection

In [9]:
LS9_SR_Mask = LS9_SR_Mosaic.map(cloud_mask)

## Change all dictionary values in 'SPACECRAFT_ID' to 'LANDSAT_9'

In [10]:
LS9_SR_Sensor = LS9_SR_Mask.map(lambda image: image.set('SPACECRAFT_ID', 'LANDSAT_9'))

## Convert the acqusition date time to a 'YYYY-MM-dd' format

In [11]:
LS9_SR_Sensor = LS9_SR_Sensor.map(
    lambda img: img.set({"Date": ee.Date(img.get("system:time_start")).format("YYYY-MM-dd")})
)

## Create a new image collection of images that are filtered out based on less than 30% cloud cover using the new 'CLOUD_COVER' percentage dictionary item

In [12]:
LS9_SR_Filtered = LS9_SR_Sensor.filterMetadata("CLOUD_COVER","less_than", 30)

## Get a count of all images in the image collection

In [13]:
# Every time the .getInfo() function is used, processing time goes up. It's a good idea to use a start and endtime on the first .getInfo() to gauge how long the others will be
start = time.time()
All_count_preprocess = LS9_SR_Filtered.size().getInfo()
print("Landsat Images: ", All_count_preprocess)
end = time.time()
print("Processing Time = " + str(round((end - start)/60, 2)) + " Minutes") # time in Minutes

Landsat Images:  51
Processing Time = 3.01 Minutes


## Get all the dates of the image collection form the new 'Date' dictionary item

In [14]:
dates = LS9_SR_Filtered.aggregate_array("Date").getInfo()
print("Dates in Imagecollection: ", dates)
#Print out the first and last dates of the created list
first = dates[0]
last = dates[-1]
print("The image collection is from " + first + " to " + last)

Dates in Imagecollection:  ['2021-11-10', '2022-01-12', '2022-02-13', '2022-02-20', '2022-03-01', '2022-03-08', '2022-03-24', '2022-05-20', '2022-06-05', '2022-06-21', '2022-07-07', '2022-07-14', '2022-07-23', '2022-08-08', '2022-08-31', '2022-09-09', '2022-09-16', '2022-10-11', '2022-10-27', '2022-11-03', '2022-11-19', '2022-11-28', '2023-01-06', '2023-03-04', '2023-03-20', '2023-03-27', '2023-04-12', '2023-06-08', '2023-07-10', '2023-07-17', '2023-07-26', '2023-09-03', '2023-09-12', '2023-09-19', '2023-09-28', '2023-10-21', '2023-11-06', '2023-12-08', '2024-01-02', '2024-01-18', '2024-02-03', '2024-02-19', '2024-02-26', '2024-03-13', '2024-03-29', '2024-04-07', '2024-04-14', '2024-04-23', '2024-06-10', '2024-06-26', '2024-07-12']
The image collection is from 2021-11-10 to 2024-07-12


## Get all the names of the space crafts in the image collection

In [15]:
sensor = LS9_SR_Filtered.aggregate_array("SPACECRAFT_ID").getInfo()
print("sensor: ", sensor)

sensor:  ['LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9']


## Function of selecting pixels of the QA_Pixel Band to mask out or preserve

In [16]:
# A function to mask out cloud shadow pixels.
def cloud_shadows(image):
    QA = image.select(['QA_PIXEL'])
    # Get the internal_cloud_algorithm_flag bit.
    return getQABits(QA, 4,4, 'Cloud_shadows').eq(0)

# A function to mask out cloudy pixels.
def clouds(image):
    # Select the QA band.
    QA = image.select(['QA_PIXEL'])
    # Get the internal_cloud_algorithm_flag bit.
    return getQABits(QA, 3,3, 'Cloud').eq(0)
       
# A function to preserve water pixels.
def water(image):
    QA = image.select(['QA_PIXEL'])
    # Get the internal_cloud_algorithm_flag bit.
    return getQABits(QA, 7,7, 'Water').neq(0) 

## Function of creating a multi mask layer to clip it out of the whole image

In [17]:
def maskClouds(image):
    cs = cloud_shadows(image)
    c = clouds(image)
    w = water(image)
    image = image.updateMask(cs).updateMask(c)
    return image.updateMask(w)

## Clip out the entire image to the dimensions of the water body boundary feature class

In [18]:
#Clip out all pixels that are in the TN River boundary and save as a new Image Collection using the lambda function
LS9_SR_Clip = LS9_SR_Filtered.map(lambda image: image.clip(TN_River))

## Run the multimask function on the entire image to remove clouds, cloud shadows, and land pixels

In [19]:
#Apply each equation to the clipped pixels that are in the TN River boundary
LS9_SR_Clip_Water = LS9_SR_Clip.map(maskClouds)

## Function that scales a bands DN values to reflectance values and applies a modeling on the pixels to convert it to desired parameters like NTU or Chlorophyll

In [20]:
#Create fucntion that calcualtes the Quantitative index 
def QUANT(image):
    red = image.select('Red') #Create a variable that selects the red band
    scale = red.multiply(0.0000275).add(-0.2)
    #Run the '2677.2 * (pow(Red, 1.856))' equation from https://www.researchgate.net/publication/354736333_Remote_Sensing_of_Turbidity_in_the_Tennessee_River_Using_Landsat_8_Satellite on page 14
    quant = (((scale.pow(1.856)).multiply(2677.2)).toFloat()).rename('QUANT') #Will have to change this if you found a different band and equation to use
    return quant

## Apply the scaling and modeling function on the preprocessed image collections 

In [21]:
LS9_SR_Clip_Water_Quant = LS9_SR_Clip_Water.map(QUANT)

## Create a list and count the number of images to be later used to export the whole filetered image collection

In [22]:
#Create a list that collects that orders the images so they can be mapped to the dates
LS9_SR_Clip_Water_Quant_List = LS9_SR_Clip_Water_Quant.toList(LS9_SR_Clip_Water_Quant.size())

## Create a for loop that exports all images to a folder in your local drive or your Google drive

In [ ]:
#QUANT iterate images to add map to layer, export to local directory, and export to Google Drive
for index in range(0, All_count_preprocess):
    image = ee.Image(LS9_SR_Clip_Water_Quant_List.get(index))
    layer_name = "QUANT_" + str(dates[index]) + "_" + str(sensor[index])
    #Map.addLayer(image, ndssiVis, layer_name, False) #Uncomment this line if you would like to see the image in the map window of the GEE map interactive
    #QUANT_file = os.path.join(r'D:\Thesis\ASPRS\GEE_Assessment\Data\Rasters\Quant_Test', layer_name + ".tif") #Uncomment this line if you would like to create a path to a local folder on your computer to export
    #geemap.ee_export_image(image, filename = QUANT_file, scale = 30, region = TN_RiverGeom, file_per_band = True) #Uncomment this line if you would like to export the data localy to the declared folder. Note: Big image exports won't work
    geemap.ee_export_image_to_drive(image, description = layer_name, folder = 'LS9_QUANT_' + first + '_to_' + last, region = TN_RiverGeom, scale = 30) #Use for big image exports. Veiw progress at https://code.earthengine.google.com/tasks